<a href="https://colab.research.google.com/github/purang2/BERT/blob/main/%ED%98%B8%EB%B0%98%EC%9A%B0NLP_ELECTRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets #from huggingface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 8.9 MB/s 
     |████████████████████████████████| 6.6 MB 55.2 MB/s 
     |████████████████████████████████| 120 kB 66.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 6.9 MB/s 
     |████████████████████████████████| 212 kB 64.4 MB/s 
     |████████████████████████████████| 115 kB 67.5 MB/s 
     |████████████████████████████████| 127 kB 71.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from datasets import load_dataset
import pandas as pd 

data_path ="drive/MyDrive/호반우NLP_한국인감정/KERC22Dataset/KERC22Dataset_PublicTest"


train_data = pd.read_csv(data_path+"/train_data.tsv", sep='\t')
train_labels = pd.read_csv(data_path+"/train_labels.csv")

train_data

,sentence_id,person,sentence,scene,context
0,1,어영,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,S0104,NaN
1,2,어영,우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,S0108,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는"
2,3,지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
3,4,순경,근무중인데 어딜가?,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
4,5,지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
...,...,...,...,...,...
7334,12285,청난,종남이 맛있는것도 사주고 잘 챙겨줘.,S3576,NaN
7335,12286,건강,걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.,S3576,NaN
7336,12287,종남,아빠. 우리 저거 타.,S3576,NaN
7337,12288,건강,"나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너도 내몸땡이에 ...",S3576,NaN


In [3]:
train_labels

,sentence_id,label
0,1,dysphoria
1,2,dysphoria
2,3,neutral
3,4,dysphoria
4,5,euphoria
...,...,...
7334,12285,euphoria
7335,12286,euphoria
7336,12287,euphoria
7337,12288,euphoria


In [4]:
train_drop = train_data[['context','sentence','person']]


train_drop

,context,sentence,person
0,NaN,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,어영
1,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는",우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,어영
2,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",지구대
3,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,근무중인데 어딜가?,순경
4,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,지구대
...,...,...,...
7334,NaN,종남이 맛있는것도 사주고 잘 챙겨줘.,청난
7335,NaN,걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.,건강
7336,NaN,아빠. 우리 저거 타.,종남
7337,NaN,"나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너도 내몸땡이에 ...",건강


In [5]:
train_pair = train_drop

train_pair['label'] = train_labels['label']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
train_pair

,context,sentence,person,label
0,NaN,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,어영,dysphoria
1,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는",우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,어영,dysphoria
2,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",지구대,neutral
3,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,근무중인데 어딜가?,순경,dysphoria
4,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,지구대,euphoria
...,...,...,...,...
7334,NaN,종남이 맛있는것도 사주고 잘 챙겨줘.,청난,euphoria
7335,NaN,걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.,건강,euphoria
7336,NaN,아빠. 우리 저거 타.,종남,euphoria
7337,NaN,"나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너도 내몸땡이에 ...",건강,euphoria


In [7]:
import numpy as np 

train_pair['context2'] = train_pair['context'].fillna('none')
train_pair['sentence2'] = train_pair['sentence'].fillna('none')
#모든 글자 합치기
train_pair['new'] = train_pair['person'] +" " + train_pair['context2'] + " " + train_pair['sentence2']
#train_pair['new'] = train_pair['sentence'] +" " + train_pair['context'] + " " + train_pair['person']

train_pair

,context,sentence,person,label,context2,sentence2,new
0,NaN,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,어영,dysphoria,none,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,어영 none 야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 ...
1,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는",우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,어영,dysphoria,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는",우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,"어영 포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는 우리..."
2,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",지구대,neutral,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",지구대 초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리...
3,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,근무중인데 어딜가?,순경,dysphoria,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,근무중인데 어딜가?,순경 초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는...
4,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,지구대,euphoria,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,지구대 초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리...
...,...,...,...,...,...,...,...
7334,NaN,종남이 맛있는것도 사주고 잘 챙겨줘.,청난,euphoria,none,종남이 맛있는것도 사주고 잘 챙겨줘.,청난 none 종남이 맛있는것도 사주고 잘 챙겨줘.
7335,NaN,걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.,건강,euphoria,none,걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.,건강 none 걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.
7336,NaN,아빠. 우리 저거 타.,종남,euphoria,none,아빠. 우리 저거 타.,종남 none 아빠. 우리 저거 타.
7337,NaN,"나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너도 내몸땡이에 ...",건강,euphoria,none,"나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너도 내몸땡이에 ...","건강 none 나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너..."


In [8]:
train_pair.loc[0,'new']

'어영 none 야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는 주제에 법을 지켜?'

In [9]:
train_pairs = train_pair[['label','new']]

train_pairs.head()

,label,new
0,dysphoria,어영 none 야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 ...
1,dysphoria,"어영 포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는 우리..."
2,neutral,지구대 초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리...
3,dysphoria,순경 초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는...
4,euphoria,지구대 초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리...


In [10]:
labels = {'euphoria':0,
          'neutral':1,
          'dysphoria':2
          }

train_pairs['label'] = [labels[label] for label in train_pairs['label']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
train_pairs

,label,new
0,2,어영 none 야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 ...
1,2,"어영 포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는 우리..."
2,1,지구대 초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리...
3,2,순경 초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는...
4,0,지구대 초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리...
...,...,...
7334,0,청난 none 종남이 맛있는것도 사주고 잘 챙겨줘.
7335,0,건강 none 걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.
7336,0,종남 none 아빠. 우리 저거 타.
7337,0,"건강 none 나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너..."


In [12]:
train_pairs.groupby(['label']).size()

label
0    1967
1     846
2    4526
dtype: int64

In [13]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_pairs, test_size=0.2, random_state=42)

In [14]:
from datasets import Dataset
import pandas as pd 




train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)




In [15]:
from transformers import AutoTokenizer

max_input_length = 200
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-discriminator")


def preprocess_function(examples):
    inputs = [doc for doc in examples["new"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    return model_inputs

#.remove_columns(["input_texts","target_texts","token_type_ids"])

train_dataset =train_dataset.map(preprocess_function, batched=True).remove_columns(["__index_level_0__",'new'])
val_dataset = val_dataset.map(preprocess_function, batched=True).remove_columns(["__index_level_0__",'new'])

train_dataset


train_dataset

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/279k [00:00<?, ?B/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5871
})

In [16]:
print(train_dataset[228])

{'label': 2, 'input_ids': [2, 2884, 5810, 3364, 5785, 5688, 5785, 2863, 10689, 5735, 27564, 18, 1073, 5787, 24092, 12050, 2884, 5810, 3388, 5662, 5896, 13510, 5668, 11230, 5733, 5, 11192, 10745, 12146, 1415, 10618, 12811, 28054, 10552, 6069, 12054, 35, 16792, 12614, 12755, 5668, 2049, 24070, 16, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [17]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-discriminator")

TRAIN 조건 (BASELINE)
- EPOCH: 30 
- learning_rate = 0.0001
- batch_size = 32
- bilstm_hidden_size = 64
- clf_out = 3

참고한 자료
- https://jimmy-ai.tistory.com/283

In [18]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 4.7 MB/s 


In [19]:
import evaluate
accuracy = evaluate.load("accuracy")

### metric: Micro-F1 Score (멀티클래스 분류에 특화)

- https://huggingface.co/spaces/evaluate-metric/f1

In [20]:
def compute_metrics(eval_preds):
    metric = evaluate.load("f1")
    #metric = evaluate.load('glue', 'mnli')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    #return metric.compute(predictions=predictions, references=labels)
    return metric.compute(predictions=predictions, references=labels, average='micro')

### Model 

- https://huggingface.co/monologg/koelectra-base-discriminator

### Train strategy 관련

- [네트워크 성능개선 전략](https://ysyblog.tistory.com/111)

- [레이블 불균형 문제 해결 방안 -> '데이터 증강' #1 - 입문편](https://shinminyong.tistory.com/34)
- [레이블 불균형 문제 해결 방안 -> '데이터 증강' #2 - 심화편](https://soundprovider.tistory.com/entry/%EB%AC%B8%EC%84%9C%EB%B6%84%EB%A5%98-PoC-Long-Document-Data-Imbalance)
- [오버피팅에 관한 글](https://stats.stackexchange.com/questions/282160/how-is-it-possible-that-validation-loss-is-increasing-while-validation-accuracy)

### 성능 향상 방안: 연구적인 접근법

- [(Blog) X-BERT: eXtreme Multi-label Text Classification using BERT](https://simonezz.tistory.com/83?category=892980)

- [(논문) GPT-2를 활용한 생성 기반의 텍스트 분류 데이터 증강법 (오 굿..!!⭐)](https://arxiv.org/abs/2009.10778)



In [44]:
#from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-discriminator", num_labels=3)


#Gather your training arguments~
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=7e-6,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=8,
    weight_decay=0.01,
)



#Collect your model, training args, dataset, data collator, tokenizer in Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset= val_dataset,
    data_collator=data_collator, 
    tokenizer=tokenizer,  
    compute_metrics = compute_metrics  
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--monologg--koelectra-base-discriminator/snapshots/c7005c19e7e523a86c96ad67fbd49c888ebbf287/config.json
Model config ElectraConfig {
  "_name_or_path": "monologg/koelectra-base-discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_typ

In [45]:

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5871
  Num Epochs = 8
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1472


Epoch,Training Loss,Validation Loss,F1
1,No log,0.842251,0.609673


***** Running Evaluation *****
  Num examples = 1468
  Batch size = 32


KeyboardInterrupt: ignored

## Evaluate


참고 
- https://dacon.io/competitions/official/235900/codeshare/4872?page=1&dtype=recent

In [39]:
data_path ="drive/MyDrive/호반우NLP_한국인감정/KERC22Dataset/KERC22Dataset_PublicTest"


test_data = pd.read_csv(data_path+"/public_test_data.tsv", sep='\t')




test_data

,sentence_id,person,sentence,scene,context
0,10,현찰,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람..."
1,11,현찰,"계약할 서류는 태연희실장 통해서 보내 드렸는데 확인해 보셨죠? 예, 예.",S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람..."
2,12,경위1,김순경 경사님 사모님 아니십니까?,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람..."
3,13,과자,아 예.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람..."
4,14,경위1,저 기억하시죠? 00지구대에서 김경사님 모시고 근무했었습니다.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람..."
...,...,...,...,...,...
2561,12256,우미현찰애들,다녀오겠습니다.,S3570,가방든 현찰우미. 애들 신났고. 순경건강 나와선. 부러운듯 보는 종남
2562,12257,건강,부럽냐?,S3570,우르르 몰려 나가는. 종남 부러운듯 보는. 순경은 들어갔고. 종남 보는 건강
2563,12258,종남,응.,S3570,우르르 몰려 나가는. 종남 부러운듯 보는. 순경은 들어갔고. 종남 보는 건강
2564,12259,건강,그럼 까짓거 우리도 놀러가지뭐.,S3570,우르르 몰려 나가는. 종남 부러운듯 보는. 순경은 들어갔고. 종남 보는 건강


In [40]:
test_pair = test_data[['sentence_id','context','sentence','person']]

test_pair['context2'] = test_pair['context'].fillna('none')
test_pair['sentence2'] = test_pair['sentence'].fillna('none')
#모든 글자 합치기
test_pair['new'] = test_pair['person'] +" " + test_pair['context2'] + " " + test_pair['sentence2']

#test_pairs = test_pair[['label','new']]
test_pairs = test_pair[['sentence_id','new']]

test_dataset = Dataset.from_pandas(test_pairs)
test_dataset =test_dataset.map(preprocess_function, batched=True).remove_columns(['new'])


predictions = trainer.predict(test_dataset)

predictions.predictions


  0%|          | 0/3 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: sentence_id. If sentence_id are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2566
  Batch size = 32


array([[ 0.47018194, -0.5052558 , -0.43804827],
       [ 0.48355955, -0.49424738, -0.463348  ],
       [ 0.49819374, -0.47490254, -0.49020848],
       ...,
       [ 0.28939813, -0.3679146 , -0.32720634],
       [ 0.37714788, -0.75007474, -0.19032833],
       [ 0.3720067 , -0.3753176 , -0.42003566]], dtype=float32)

In [41]:
predictions.predictions[60:140]

array([[ 2.4823140e-01, -9.7854549e-01,  3.1291410e-01],
       [ 1.0439298e-01, -8.8892370e-01,  3.9355502e-01],
       [-4.2538738e-01, -1.4122517e+00,  1.7812537e+00],
       [-4.9713975e-01, -1.4842768e+00,  1.8742572e+00],
       [ 3.4602007e-01, -8.0038643e-01, -5.3505041e-02],
       [-3.4564978e-01, -1.4215053e+00,  1.6241573e+00],
       [-5.1847994e-01, -1.4507738e+00,  1.8633388e+00],
       [-4.5283544e-01, -1.4108694e+00,  1.8746197e+00],
       [-2.9362041e-01, -1.1262017e+00,  1.1864396e+00],
       [-2.7658039e-01, -1.3684191e+00,  1.4110392e+00],
       [ 4.1687626e-01, -4.0158936e-01, -3.4525776e-01],
       [ 4.0670443e-01, -4.9663997e-01, -2.4705622e-01],
       [ 7.3595762e-02, -1.3677306e+00,  9.2172915e-01],
       [ 3.4670955e-01, -8.9012539e-01,  9.6290775e-02],
       [ 3.3226460e-01, -7.5096506e-01, -1.1443731e-03],
       [-2.3378983e-01, -1.4746609e+00,  1.5921041e+00],
       [ 4.2033648e-01, -5.9696561e-01, -2.5968719e-01],
       [ 4.0840352e-01, -4.9251

In [42]:
test_dataset

Dataset({
    features: ['sentence_id', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2566
})

In [43]:

list1 = []
list2 = []


for i in range(2566):
  list1.append(i)
  list2.append(-1)



raw_data = {'Id': list1,
            'Predicted': list2
            }
df = pd.DataFrame(raw_data)



labels = {'euphoria':0,
          'neutral':1,
          'dysphoria':2
          }

df['Predicted'] = np.argmax(predictions.predictions, axis=-1)

d = ['euphoria','neutral','dysphoria']

df['Predicted'] = [d[label] for label in df['Predicted']]


df['Id'] = test_dataset['sentence_id']

#x = [d[labels[label]] for label in ['euphoria','euphoria','dysphoria','neutral']]

df


df.to_csv('drive/MyDrive/호반우NLP_한국인감정/submission_no3.csv', index=False)

In [ ]:
df

In [ ]:
df['Predicted'].value_counts()

In [ ]:
['dys','neu','eup'][2]